# Exploratory Data Analysis of rental table

In [19]:
import pandas as pd 

In [20]:
rental = pd.read_csv("../data/output/rental.csv")

rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     16044 non-null  int64 
 1   rental_date   16044 non-null  object
 2   inventory_id  16044 non-null  int64 
 3   customer_id   16044 non-null  int64 
 4   return_date   15861 non-null  object
 5   staff_id      16044 non-null  int64 
 6   last_update   16044 non-null  object
dtypes: int64(4), object(3)
memory usage: 877.5+ KB


### Missing data

Appears to be null values in the `return_date` column
My hypothesis is that the nulls in `return_date` are rentals which are *yet to be returned* but which are still valid purchases

To find out let's find the average length of time between `return_date` and `rental_date` and identify the most recent record, to find out if the nulls are rentals which are yet to be return, or rentals which are **LOST** 


In [51]:
from datetime import datetime, timedelta

# How  many null values are there in return_date
missing_returns = rental["return_date"].isnull()
print(f"Total missing returns {missing_returns.sum()}")

# Average and max time between the rental_date and return_date

rental["rental_date"] = pd.to_datetime(rental["rental_date"])
rental["return_date"] = pd.to_datetime(rental["return_date"])
rental["last_update"] = pd.to_datetime(rental["last_update"])

average_duration = (rental["return_date"] - rental["rental_date"]).mean()
print(f"Average time to return: {average_duration.days} days")

# Not including any unreturned rentals to only include actually returned records
max_time_to_return = (rental["return_date"].dropna() - rental["rental_date"]).max()
print(f"Greatest time to return: {max_time_to_return.days} days")

# Most recent record 
newest_record = rental['last_update'].max()
print(f"Most recent date: {newest_record}")

# Should the missing return_dates have been returned 

unreturned_rental_dates = rental.loc[rental["return_date"].isnull(), ["rental_date"]]
average_date_unreturned = unreturned_rental_dates.mean()
time_since_unreturned_rentals = (newest_record - average_date_unreturned)

print(f"Average rental date of unreturned: {average_date_unreturned}")
print(f"Time since unreturned rentals recorded : {time_since_unreturned_rentals}")




Total missing returns 183
Average time to return: 5 days
Greatest time to return: 9 days
Most recent date: 2006-02-23 09:12:08
Average rental date of unreturned: rental_date   2006-02-13 15:58:25.453552256
dtype: datetime64[ns]
Time since unreturned rentals recorded : rental_date   9 days 17:13:42.546447744
dtype: timedelta64[ns]


> Since it is ambiguous whether or not the unreturned rentals (~1% of the data) are LOST or yet to be returned, to normalize data I will change the values to assume they will be returned after 9 days - the max time recorded. 

### Duplicates


In [53]:
rental.duplicated().sum()

np.int64(0)

It is apparent there are no duplicated records in the rental table